In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
tf.reset_default_graph()

In [4]:
# Hyper Parameters
learning_rate = 0.01   
n_steps = 28         
n_inputs = 28     
n_hiddens = 64     
n_layers = 2       
n_classes = 10   

In [5]:
mnist = input_data.read_data_sets("./dataset/", one_hot=True)
test_x = mnist.test.images
test_y = mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./dataset/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./dataset/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./dataset/t10k-images-idx3-ubyte.gz
Extracting ./dataset/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# Input

In [6]:
x = tf.placeholder(tf.float32, [None, n_steps * n_inputs], name='x_input')  
y = tf.placeholder(tf.float32, [None, n_classes], name='y_input')        
keep_prob = tf.placeholder(tf.float32, name='keep_prob_input')      
batch_size = tf.placeholder(tf.int32, [], name='batch_size_input')  

# Weights and Biases
#### Prepare for the fully-connected layer

In [7]:
with tf.name_scope('weights'):
    Weights = tf.Variable(tf.truncated_normal([n_hiddens, n_classes],stddev=0.1), dtype=tf.float32, name='W')
with tf.name_scope('biases'):
    biases = tf.Variable(tf.random_normal([n_classes]), name='b')

# RNN Structure

In [8]:
def RNN_LSTM(x, Weights, biases):
    x = tf.reshape(x, [-1, n_steps, n_inputs])
    def attn_cell():
        lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hiddens)
        with tf.name_scope('lstm_dropout'):
            return tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
    enc_cells = []
    for i in range(0, n_layers):
        enc_cells.append(attn_cell())
    with tf.name_scope('lstm_cells_layers'):
        mlstm_cell = tf.contrib.rnn.MultiRNNCell(enc_cells, state_is_tuple=True)
    _init_state = mlstm_cell.zero_state(batch_size, dtype=tf.float32)
    outputs, states = tf.nn.dynamic_rnn(mlstm_cell, x, initial_state=_init_state, dtype=tf.float32, time_major=False)
    return tf.nn.softmax(tf.matmul(outputs[:,-1,:], Weights) + biases, name='z_output')

In [9]:
pred = RNN_LSTM(x, Weights, biases)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


# Cost

In [10]:
with tf.name_scope('loss'):
    cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred),reduction_indices=[1]))

# Optimizer

In [11]:
with tf.name_scope('train'):
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy

In [12]:
with tf.name_scope('accuracy'):
    accuracy = tf.metrics.accuracy(labels=tf.argmax(y, axis=1), predictions=tf.argmax(pred, axis=1))[1]

# Run and Save the Model

In [13]:
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

In [14]:
CKPT_DIR = 'new_ckpt_dir_4'

In [15]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    saver = tf.train.Saver(max_to_keep = 0)
    for i in range(2000):
        _batch_size = 128
        batch_x, batch_y = mnist.train.next_batch(_batch_size)
        _, loss = sess.run([train_op, cost], feed_dict={x:batch_x, y:batch_y, keep_prob:0.5, batch_size:_batch_size})
        if (i + 1) % 100 == 0:
            print('No %6d step, current loss: %.3f'%(i+1,loss))
    saver.save(sess,CKPT_DIR + '/model',global_step = 2000)
    print("Optimization Finished!")
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x:test_x, y:test_y, keep_prob:1.0, batch_size:test_x.shape[0]}))

No    100 step, current loss: 0.977
No    200 step, current loss: 0.367
No    300 step, current loss: 0.310
No    400 step, current loss: 0.271
No    500 step, current loss: 0.181
No    600 step, current loss: 0.111
No    700 step, current loss: 0.148
No    800 step, current loss: 0.146
No    900 step, current loss: 0.050
No   1000 step, current loss: 0.071
No   1100 step, current loss: 0.150
No   1200 step, current loss: 0.104
No   1300 step, current loss: 0.086
No   1400 step, current loss: 0.063
No   1500 step, current loss: 0.099
No   1600 step, current loss: 0.210
No   1700 step, current loss: 0.078
No   1800 step, current loss: 0.066
No   1900 step, current loss: 0.160
No   2000 step, current loss: 0.069
INFO:tensorflow:new_ckpt_dir_4/model-2000 is not in all_model_checkpoint_paths. Manually adding it.
Optimization Finished!
Testing Accuracy: 0.9792
